# Libraries      
---

In [1]:
# 기본
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings("ignore")
import re

# 시각화 
import plotly.express as px
import matplotlib.pyplot as plt 
import seaborn as sns

# 모델
from sklearn.model_selection import (train_test_split, 
                                     KFold, 
                                     StratifiedKFold,
                                     GridSearchCV,
                                     RandomizedSearchCV)

from sklearn.tree import (DecisionTreeClassifier, 
                          DecisionTreeRegressor)

from sklearn.ensemble import (RandomForestClassifier, 
                              RandomForestRegressor)

from sklearn.svm import SVC

from xgboost import XGBClassifier

# sampling
from imblearn.over_sampling import SMOTE
from collections import Counter

# report
from sklearn import metrics
# matrix = metrics.confusion_matrix(y_test, y_pred)
from sklearn.metrics import classification_report, roc_curve, auc, accuracy_score

#encoder, scaler
from sklearn.preprocessing import (LabelEncoder, 
                                   OneHotEncoder, 
                                   StandardScaler,
                                   MinMaxScaler)

# imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# test
import statsmodels.api as sm 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# stopwords
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy import tag
from wordcloud import WordCloud

kkma = Kkma()


In [2]:
plt.rcParams['axes.unicode_minus'] = False
custom_params = {"axes.spines.right": False, "axes.spines.top": False}

sns.set_theme(context = 'talk',  
              style = 'darkgrid',  
              font = 'Malgun Gothic', 
              font_scale = 1.2,         
              rc = custom_params) 

# Read the Data         
---

In [3]:
cur_dir = os.getcwd()

for root, dirs, files in os.walk(cur_dir):
    print(root, files)

c:\Users\user\Documents\b7_Crawling\김재경 ['.gitignore', 'data_handle.ipynb', 'incruit.ipynb', 'merge_combine.ipynb', 'New_project.ipynb', 'rogistic.ipynb', 'SMINFO.ipynb', 'stop_words_test.ipynb', 'test.xlsx', 'wanted_final.ipynb', 'wanted_insight.ipynb', 'wanted_test.ipynb', 'word_cloud.ipynb', '매출액전처리.ipynb', '시각화 (1).ipynb', '시각화.ipynb', '카이제곱.ipynb']
c:\Users\user\Documents\b7_Crawling\김재경\.vscode ['settings.json']
c:\Users\user\Documents\b7_Crawling\김재경\data ['data_res.csv', 'Untitled-1.ipynb', '발표자료.txt', '추가분석.txt']
c:\Users\user\Documents\b7_Crawling\김재경\data\.gitignore ['company_info.csv', 'corp_data_1.xlsx', 'corp_data_2.csv', 'data_result.csv', 'incruit_result.xlsx', 'wanted_dart_data.csv', 'wanted_data.csv', 'wanted_data_result.csv', 'wanted_incruit_data.csv', 'wanted_insight_data.csv', 'wanted_result.csv', '데이터 전처리.ipynb']
c:\Users\user\Documents\b7_Crawling\김재경\data\.gitignore\handle_data ['wanted_dart_handle.csv', 'wanted_incruit_handle.csv', 'wanted_insight_handle.csv']


In [4]:
data_list = os.listdir(cur_dir + "\data")
Dataset = dict()

for data in data_list:
    if ".csv" in data:
        Dataset[data[:-4]] = pd.read_csv(cur_dir + f"\data\{data}", index_col = 0)

In [5]:
data = Dataset['data_res']
df = data.copy()

# Data info        
---

In [6]:
df.head()

,기업명,업종분류,사원수,설립연도,기업형태,매출액,주소,직무,자격요건,직급,이용하는기술스택/우대사항,해당 페이지 URL,기업 홈페이지 URL,수요
0,젭,데이터베이스 및 온라인정보 제공업,NaN,2021.0,NaN,NaN,서울특별시 강남구,소프트웨어 엔지니어,"프로그래밍언어,열정,성실",3,"멀티,쓰레드,서버,웹,웹소켓,소켓,클라우드,구축,메타버스,AWS",https://www.wanted.co.kr/wd/180861,NaN,NaN
1,서커스컴퍼니,응용 소프트웨어 개발 및 공급업,20.0,2012.0,중소기업,170563.0,서울특별시 강남구,소프트웨어 엔지니어,"전공무관,공무,Node,Net,Nginx,Apache,SystemDesign,MyS...",0,"소프트웨어,객체지향,배포,자동화,프로젝트,서버,아키텍처,버전관리,AWS,EC,S,C...",https://www.wanted.co.kr/wd/207284,www.circuscompany.com,NaN
2,블로코,응용 소프트웨어 개발 및 공급업,29.0,2014.0,중소기업,118214.0,경기도 성남시 분당구,소프트웨어 엔지니어,"개발경력2년,대용량데이터,데이터,Golang",2,"블록,블록체인,체인,트랙트,프레임워크,개발경험,암호화,전자서명,클라우드,spring...",https://www.wanted.co.kr/wd/206690,ww.blocko.io,NaN
3,피매치,자연과학 및 공학 융합 연구개발업,NaN,2021.0,NaN,5286.0,서울특별시 성북구,소프트웨어 엔지니어,"서비스개발,데이터,딥러닝프레임워크,AI,Python,Tensorflow,Pytorch",2,"헬스케어,자연어,모델,영문,논문,분석,LLM,Linux,Docker",https://www.wanted.co.kr/wd/186806,www.pmatch.co.kr,NaN
4,만나플래닛,응용 소프트웨어 개발 및 공급업,134.0,2016.0,중소기업,1419493.0,서울특별시 구로구,소프트웨어 엔지니어,JAVA,5,"협업,프레임워크,트렌드,Rest API,API,Git,redmine",https://www.wanted.co.kr/wd/206244,www.manna-planet.com,NaN


In [7]:
df.columns

Index(['기업명', '업종분류', '사원수', '설립연도', '기업형태', '매출액', '주소', '직무', '자격요건', '직급',
       '이용하는기술스택/우대사항', '해당 페이지 URL', '기업 홈페이지 URL', '수요'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1965 entries, 0 to 1964
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   기업명            1965 non-null   object 
 1   업종분류           1927 non-null   object 
 2   사원수            1014 non-null   float64
 3   설립연도           1927 non-null   float64
 4   기업형태           1112 non-null   object 
 5   매출액            1769 non-null   float64
 6   주소             1948 non-null   object 
 7   직무             1963 non-null   object 
 8   자격요건           1945 non-null   object 
 9   직급             1965 non-null   int64  
 10  이용하는기술스택/우대사항  1958 non-null   object 
 11  해당 페이지 URL     1965 non-null   object 
 12  기업 홈페이지 URL    1894 non-null   object 
 13  수요             0 non-null      float64
dtypes: float64(4), int64(1), object(9)
memory usage: 230.3+ KB


# Stopwords

In [9]:
df = df.drop(columns = ['해당 페이지 URL', '기업 홈페이지 URL', '수요'])

In [10]:
data = df

In [11]:
# data['이용하는기술스택/우대사항'] = data['이용하는기술스택/우대사항'].fillna(1)

# ll = ''
# for www in data['이용하는기술스택/우대사항']:
#     if www != 1:
#         ll += ''.join(www.replace('•',' ').replace('ㆍ',' ').replace('등',' ').replace('가능한자', ' ').replace('하신분', ' ').replace('해본경험', ' ').replace('해보신분',' ').replace('+\d만원',' ').replace('+\d억',' '))
#     else:
#         pass


# stop_words_1 = '''이해 분 관심 웹 스타트업소통 협업 대치 대치동 라즈베리 센서 열의선정 선정 미래 10명 서치콘솔 강한 면제 군면제 웹성능 의무 남성 응용경험 밑바닥 조절 장애대응 프리세일즈 예방 재개발 협력사 검색서비스 기업데이터 엑 엑셈 어필 스타일링 메시징 해당직무 데이트 난이도 에너지골프 선정 검색엔진최적화 여행시스템웹 적극성단위테스트 노하우측정 기여경험여 토큰화 오아시스 바리 멤버십 업무해본 2시간 튜드 실행력 기능테스트 개발자자발적 100만원 종단테스트 종단 토큰증권 앰플 공공 리액트로 국가사업 맨 마인드개발자 해치랩스의 카메라 업그레이드 토큰 팀빌딩 리딩해본 유사언어 폴리곤 쇼핑물 랩스 피칭 업그레이더블 데스크톱 데스크톱앱 출력 학습라이브 인터랙션 성실 워크로드 지속적인 성능향상 측면 공공기관 지식구축 언어권 베스트 거리낌 적성 중급 방어 방어기술 노하우플랫폼 스타트업의 기여경험 사이버 프론트성능 공공데이터 개방 프로젝트전문 논술 노드 포트폴리오대용량 마음운용 활용경험 설계자 가격 원칙 준수자 음악유통정산 유통 참여경험 차키 축구 테크성능 인하우스 계정 학술지 솔루션인공지능 벡터 개발이력 복리후생비머스 자사 자사상품 커뮤니케이션배포 만난 반대 기한 토끼굴 배려심 외주 예약서비스 하우스 토대 록체인 지향점 음악 레벨 협업스킬 통용 비기능 먼트 간부블록 샷 애착 가상자산 자산 데일리 차원 이중화 파인 문서관리시스템 매사 계발 효율성 검증 아시 동기부여 업무영역 한계 팬덤 성장과정 학습프로젝트 적합성 정확 호텔예약엔진 컨벤션 이해도과 게임서버 소프트스킬알고리즘 성능최적화 브랜치 발생시 에이전시 부하테스트 싸이클 소규모 한정적 여행시스템데이터 열의 지식애니메이션 기능단위 서버관리 화의 겅 데이터기반 열의배포 커스텀 자유형식 성장유관 팩터링 물류시스템 정산 키택 자기개발상용서비스 독립적 롱위켄드 태스크 애니메이션 합격자 공학계열 업무용 리조트 유지보수성 서치 자발적 뮤케이션 고도화최적화 스크럼 실서버 백오피스 팀웍블록 망설임 생태계 항공예약엔진 호텔 예약 주니어 탐구심 줄 전시전자공학 사이드 고객중심 유니티3 단위테스트 커뮤니케이션데이터 디자인시스템 원리 템플 릿 필요성 점진적 웹앱 분산관리 라즈베리파이 사용경험자 2호선 개발자이지만 이지만 웹소설 자극 소설 허들 운용 심리적 신뢰성검사 선릉역 적재 물리적 수집 기본지 프런트 신뢰성 비개발조직 리더쉽 개발경력플랫폼 개발자서버 무중단 인근 과의 대주주 횡포 배포관리 결격사항 지식배포 디자인팀 주식 웹아키텍처 운영관련 포트포워딩 워딩 100만건 코드버전 발표 인트라넷 커뮤니케이션광고 타고 광고플랫폼 고도몰 영카트 카트 정보통신공학 데이타고 커뮤니케이션협업 기초블록 환경자 대인관계 의사결정 콘솔 활동적 관련전공자 굿스플 노하우 데이터링 스포츠 원인 교육기관 희망 희망자 이수 제자 양성보안 실력 프론트모듈 흐름제어 수상 비개발조직 주도적쇼핑물 레플리카 비법 세일 즈맵 마음앱 시니어 편입 전문연구요원 세일즈맵 코드품질 융합 21 골프 안주 운영환경 관장 에이 속도 분산처리 헤더 고성능 대량 캐시 대치 기술조직 개발조직 웹표준  대치동 무선충전 이전 전문적 정부 툰 이슈관리 소셜카지노 유관업무 툰 지사 부천지사 부천 거부함 기술도입 개발문화 작용 상호작용 자일 이공 전공자이공 기사 협업역량 성장커뮤니케이션 선릉역 소프트스킬상태관리 세서 점심 구비 도전정신 컨텐츠 시나리오 가설 아이폰 이타 동료스토어 능력플리케이 국가유공자 고용촉진지원금 촉진 보신분 처리해본 지침 개인정보보호 인증업무 번역 유관분야 셈 일치 전시솔루션 친화력카지노 장기간 테스트케이스 기구 코드베이스 질량 통제 제약사 추출 게임업계 부탁드립 드립 해결책자동화 보드게임 정립 품질관리 전업 사교적 겸험이 험이 행동로그 스토리 유저향 내부통제 10인 바이오업계 기술특허 수강 구조화 협업도구 전시역략 프로젝트신약 질량분석 연구개발  헬스케어 전자서명 흐름도 주도성 내외부 플라이 스에 점검 일어 일로 계약 영향평가 입사확정 순서 입사일로 일로 계약 무역량 실무역량 객체지향 의욕 멘토링 획득 재무 -24 시트 스타일시트 양성패턴 유경험자패턴 능력콘텐츠 이론콘텐츠 물리 스토리텔링 간결함 무리 밖 회피 완벽 상화 계좌 금융기관 감지 우신분 지라 주일 1주일 전과정 공동의 비스니스 오브젝트 반응형 학부 페어 출판 피드백 컴퓨팅 세팅 퍼블릭 해결책 시용기간 웹사이트 서빙 알고리즘 데이터구조 외국어 크루 교육경험 라이브러리 웹서비스 중국어 개발환경 앱유닛 디자인 웹최적화 사용기간  학습공급망 물류산업 브로커 대응 부동산 부동산시장 콘텐츠 석박사 컴퓨터과학 인프라 관련경험 형상관리 군전역 전역 갈망 신규서비스 스페이스 국어 구현경험 플레이어 혁신적 공부 듀테크 부채 밍 주도적쇼핑몰 간부 간부런칭 협력 협력툴 정책 앱영어 성격 상호 배려 그림 헌신 거래소 증권사 물류서비스 연동경험 지그재그 사방 오토 생산성 적응력 개발문서 문서 집계 개별 보훈 보훈대상자 복구 학력 미터 미터함수 동시성 크로스플랫폼 주도적인 과제전형 운영경험 32 두이 두이노 고도화 플러그인 지식온라인 생산 생산적인 어려움 퀄리티 주인의식 프리 향상 3.0 외부 보고 욕구 기록 서로 커뮤니티 심층 제시 가치관 이후 개별적 완료 전산학 인덱스 백업 프리미티브 체인긍정적 호환성 크로스 브라우징 터보 레포 지식원격 원격 인터페이스 케이스 파트너 트래 백로 진행과정 운영플랫폼 적합 전적 유닛테스트 자세 도전적 나열 장점 리더십 팔로 팔로워십 밸런스 소스분석 앤드 에너지복잡 복잡 데이터셋 비정형 메인 메인테이너 이너 2인 암호학적 학적 클린코드 파일링 퍼포먼스 이벤트 동향 리소스 확장성 기초 레이션 스트레이션 능통자 통자 입사지원서 마음 집중 기술적 부분 돌파 패키지 국제화 팀단위 모빌 빌리티 처리개념 타인 컴포넌트 비디오 배치 젠 킨스 액션 토스타입 손코딩 정보기술 팀간의 헥사고날 모듈 시각화 텍스트 탐지 부하 스토어 채팅 영상통화 통화 스터디 성능개선 구체적 토스플레이스 개발자실시간 모니터링 구축경험 함수 전문가 공개적 제품팀 린스 공학과 블랙박스 로그 유튜브솔루션 보안관제 디버그 국제학술대회 예측 파악 하이브리드 자기주도적 파이프라인 타이핑 스케일 스케일업 압축 화질측정지표 유지보수 직관적 실용적 내시 성공적 테스트코드 기술스택 인증심사 심사 보안장비 스크립트 소셜 시야 학회 저자 리눅스학습 인증심사원 대용량서비스 영상처리 연구활동 브라우저 협업툴 멀티미디어 동영상압축포맷 포맷 주관적 개관 화질 사이언스 문서화 팀워크 월드 상거래 비지니스 인터넷 인터넷사업 정보보호 법률적 전력 프로세싱 플리케이션 의료분야 저널 경량 샤시 운전가능 시뮬레이션 자차 박사과정구글 페이스북 제품군 품질향상 셋팅 알람 사이클 회화 문제해결역량 트레이서 열정적 강연 강의 교수 백신 드라이버 치트 제어 개발사 경진대회 다종 박사학위 박사과정패턴 라이브 아바 커스 커스터마이징 코딩 코딩테스트 신중 방법론 비관계형 경력자 지식석사 석사학위 영어회화 병렬 트랙 레코드 지리정보체계 이해관계자 관계자 오너쉽 오너십 기획자 스프린트 프로그램 바이오 동종업계 표준화 확장 유니티 제어기 차량연동 그래픽스 자율주행 능통 체계적 동영상 전력시장 정보통신 수학 엔진 저장소 학사 근거리 근거리무선통신 시계열 영상 팀웍 인더스트리 석사 테이블 오픈마켓 마켓 주문 중개 개선점 선점 엔드유저 개선안 아이디어 논리적 분산 신용평가모델 임상 기계 기계공학 전기관련 자격증 주요서비스 유경험자 튜닝 리포팅 비동기 병역 병역특례 특례 보충역 취약점 동아리 해킹대회 카카오 테크이미지 동료 시너지 엔지니어링 의사소통 수평적 자기개발 자료구조 운영체제 아이 프로그래밍 가상 리딩 티스레드 마인드셋 네이버 신발 패션 의류 업무방식 창의적인 토론 자동화해본 마케팅 리터 키오스 학습 주도적 전산 계열 주도적데이터 도메인 프로덕트 해결법 깃 형식 동기 스펙 코드리뷰 허브 론칭 기술기반 규격 공인 메모리 전화 공인인증 마이크로 대기업 졸업자 팀원 지도플랫폼 금융데이터 모델링 재생에너지 문제점 향상 커뮤니케이션 리더 마인드  멀티 쓰레드 서버 개발경험 모델 영문 논문 분석 주세 이유 요 토끼 굴 피그 교체 업소 개발자문제해결 아가 유행 해소 스타으업의 3차 적극성 시간동안 해치 지갑 출하신 업의 호출 사용중인 대시 고민해본 보여 보여주실 명령어 보드 소켓 열린 재충전 외국 2년이상 확산 고급 분리 액 트로 변경해본 기여겸엄혀 멘 설명서 개발주기 식별 고급사용 가이드라인 채용전형 1차면접 인성검사 2차면접 실천 혹 곤 비트 럼 그레이 더블 응답 시간투자 계신 노고 로드 드리고 안내드리고 분의 심도 소정의 후보자분의 미션 스의 역할수행 기간동안 추하신 3시간 대해 1차면접 채용전형 인성검사 결함 오류 버그 주시 책임 기념품 안정적 책임 주도적 포상휴가 취미 책임감 능동적 긍정적 최적화 사용기간사내동호회 가족 생일날 증진 부정적 심리 상품권 반차 사외 직무교육 확대 어학 편리 어린이집 휴향시설 운동비 재발 재발방지대책 방지 대책 수립 적극적인센티브 복리후생비대 성장디버깅 윈도 측정치 구축 배포 마우스 대외 대외서비스 간편 간편결제 현업 탐색 한편 사전질물 입사안내 네이션 운항 입상자 입상자자동차 성장 금융 미림 미림타워 지역 대형 4주 메신저 이기 기종 흉 지식보유 노환경 과장급 악석 경력3년이상 여자 영어가능자 입상자자동차 입상자 운항 입사안내 출장가능자 출장 이중 복수 유튜브프로메 테우스 우스 근무지역 경력직 채용유형 인원 채용인원 채용정보 파일형태 기타사항 미스 복수 복수전공 훈련비클라우드 이중전공 정의 적극적 운영중인 해결 비대칭 제작 혁신 기획 지식실시간 지도 중계 프로토 지식 온라인 광고 업자 클린 처리 항상 런칭 효율적 근본적 호기심  프론트 엔드 전문 다양성 추진력 글로벌 테크 디지털  차량 탐구 성능 사용기간 구성원 조직문화 핵심가치 진보 변화 개념적 도전 목표 업계 신기술 완성도 블록 체인 데이터 소프트 수정 메시지 품질 노력 서류검토 포트폴리오 역량 능력 보완 개선의지 사용중 숙달 성찰 바꾸기 채용절차 환경내 경내 듀 진단 내외 오프라인 스트레스 평판 평판조회 조회 결산 텔 최종합류 처우협의 기술과제 전달 주차별 차별 지연 직접 캡 연계해본 경험해본 8년 19 시차 4대보험 주5일 릉 랏 방문 정비 스플 적적 긍적적 기기 긍 코드 헬스 제로 빌드 5호선 화곡역 5분 10분이내 럭 난독 하위 호환 100만 폰 5일제 일제 17 강서 강서구 화곡동 건전 1회 사내 저녁 후생비 저녁식대 야간 생일 결혼기념일 분당 스마트 변경 시장 27701 유기 5인 실시 활성화 점수 성남 성남시 분당 촌 연세대 연세대학교 2곳 곳 분당 분당구 판교 723 404 404호 서대문 서대문구 연세 광 광혜관 라온 메디 스테이지 차피 4918 회수 의의 동반 축하 부모 경조사비 조사비 자율복장 복장 회사콘도 콘도 육아 워크샵 샵 격년 테니스 조사용품 용품 포상 10년 포상금 여름 부서별 회식비 음료 각자 개개인 주인 굿 준수 좋아 7년 무엇인가 인가 콘 솔 다루 커피챗 브렌 전략 장기근속 근속 장기 서울시 사용기간 소유자 보유자 2가지 스타일 팬 덤 분실무 견고 분우대사항 달성 지속가능 기법 존중 결정 근무환경 포구 중간 활동  중간관리자 관리자 직장 요원 더쉽 가지신분 근거주자 주자 장기근무가능자 대인 직업 훈련 기관 직업훈련기관 효과적인 인재 인재영입 영입 매니저와의 와의 주의 주의사항 동일 복리 복리후생 후생 일반적 운동 행복 행복하자 하자 몸 두시 기업내 커리어 여의대 108 파크 파크원 타워1 38 38층 상위 연봉 오전 11 11시 생략 연결 쇼핑 퇴직금 연차 연차휴가 모성 모성보호휴가 경조사 경조휴가 경조금 조금 화환 효과 피해자 오신 최선의 선의 대안 지인 시일 담당자 연락 방안 컨트리 컨트리뷰 무관 비용절감 절감 부담 휴양 핀 브라 우징 부탁 방법 위주 수행업무 개인홈페이지 홈페이지 1시간 정식 암호 화폐 어린이 슬픔 화곡 기쁨 장례 지인분 1년 여정 비어 90 90분 스킬 인상 셰어 라운드 6개월 있음 최소화 주민 주민번호 번호 개인정보 삭제 요함 중단 어제 오늘 이것 선배 설명 표현 안정성 삶 몰입 종속 일정 산정 2회 포지션 편안 출근버스 대중교통 운행 주택 주택자금대출 자금 통근 공간 단체보험제도 3인이상 경청 보완 사기 열정 포토 파이어 확정 주기 즉시 용인 원활 동작 용인시 수지구 09 상여금 업무우수자 기혼 기혼직원 식사 청년 내일 음공제 공제 18일 미사용시 사용시 수당 상금 임직원 건강검진비 보조 소개서 옵션 톤 해커 참여해본 전직 모두 현역 불가 본인만 바른 예정경력기술서 기술서 필요시 셜 3인 습관화 현지 메시 회복 의료비 단체보험 차이점 지가 임원진 대신 과제테스트 식비 10만 상당 건강비 교육비 정기 리프 리프레쉬 쉬 독서 독서모임 모임 시즌 멤버쉽단체 고객사 조화 주차 이자지원제도 이자 주거 대출 유의사항 최종결과 전월세 모집 조기 동의 허위사실 비지능 구매 부족 실적 모노 회의 공감 답변 명령 미들 설치 숙박 학점 학점은행 은행 성취감 선호해 호해 벡 문화적 검사 추가 유연 유연근무제도 제도 셋째 셋째주 금요일 1시 넷째 넷째주 월요일 출근 롱 롱위켄트 켄드 야근 식대 심야 교육지원 2년시 채널 튜브 관제 대한 계속 랍격자 수시 마감 3년이상 365 보수성 친숙 복지비 휴가비 통신비 종합건강검진 이해심 모범 애정 순위 엣 귀하 사회 이바지 라도 분1 만들기 13버전 엘 스틱 리초트 급여보장 초과 링크 뮤 우수자 버전 중앙 습득 참가 학술 게재 파이프 준비 요구사항 단계 추천해 공통 해당 국가 유공자 장애인 취업 취업보호대상자 대상자 관계법령 법령 7일 며칠 헥 날 시작 혜택 개선해본 현재 이내 이메일 간의 리팩 토스 플레이스 함 창업 처우조건 포스 타입 내규 1일 8시간 수습조건 근무장소 장소 4길 강남역 1번 출구 2분 지원시 참고 참고사항 모든 권장 성공 사례 포함 현장 손 소정 참여비 간소화 4년 성과 블랙 박스 자격 취득자 투브 이외 라우드 워크플 넷 신규 체계화 장려 석식 신청 커피 무제한 동급 장비 제품 퇴근 교통비 연말 겨울 방학 별도 승인 워라 보장 재택근무 비용 도서 구입 수업 환경 케어 중시 펀 마크 비전 중요성 신분 출퇴근 근무시간 조정 동시 해외여행 결격 의사 관적 유사 유지 보수 지면 264 탑재 보호 사원 우수 기본적인 지급 결격사유 시용 사유 운전 글 페이스 북 4인 전체 접속 2,000 2,000명 수용 뉴 럴 자율 주행 지리 체계 카페테리 테리 건강 검진 경조 수원 위치 고용 고용형태 검출 추적 86 64 버싱 전형절차 면접 제출서류 사지원서 지원서 양식 명절 선물 동호회 종합검진 독감 독감주사 마이 제외 모습 부여 센터 센터필드 필드 타워 교육장 경기 경기도 우대요건 요건 규모 소지자  13482주사 국내 출산 자녀 입학 포 비즈 니스 요청사항 후속 영업 끈기 관점 소유 도입 필요 경진 검수 반영 인식 지원자 안식 휴가 지원금 6년 9 9년 근속 현금 복지 포인트 편의 시설 녀 수면 수면실 간식 휴식 카페 성향 추구 결과물 연구팀장 팀장 인력 분 지향 작성 버스 재직 근무요일 체제 부서 우선 정리 읽기 쓰기 구상 독성 유니 영역 의식 전장 26262 특성 규칙 중점적 지표 유의미 고가 용성 졸업 수자 역할 번 관련교육 2년 신용 장애 산업과 수자1 4 분관련학과졸업 상응 동종 전사 유의 리드해본 구분 화면 오너 십 본인 파트 지능 발전 편집 욕망 거리 기타 크립 서비스여 혼자 집 벨 벨루가 루가 큐칙 주류 산없과 기능 분과 대상 인공 제안구애 주요 작업 시험 일부 최소 3개 1개 플 케이 인더스 트리 요구 운 영해 운영해 가능자1 너리 취약 뿐 중요시 빅 상태 비 실행 반복 레드 요청 일상 파 파티 응용 셋 진심 기업 실제 대체 분위기 자기주 다량 만족 중요 여기 실패 창의 방향 구조적 활용해 용해 각종 주도 자기 사이트 종사 수준 맥락 통합적 설계해본 요일 극복 후보자 치열 논의 수준의 커버 리지 다수 유저 오스 결제 플랫 펌 드로 이드 추천 다각 다각도로 도로 오피스 총괄 참석 전반적 핏 내부 상황 변동 내용 스스로 설득 시간 체크 합류 종합적 영향 필수 주소 선택 형시 1로 로 최대 등기 확인 앞 기대 실무진 대면 60 60분 가량 중심적 이야기 자리 시도 5일 일 40 40시간 7시 30 30분-10시 -10 30분 처우 보상 개인 바탕 협의해 절차 입사 입사지원 지원 재직해본 형성 누비 랩 개발팀 8 다음 션 근무지 서울특별 서울특별시 625 덕명 덕명빌딩 6층 삼성 삼성역 도보 3분  최고 장벽 기준 사업부 자체 프로 동안 급여 100 지금 평가 연장 종료 과정 제출 증빙 증빙자료 자료 사실 허위 기재 발견 취소 콜 8 론 재미 직군 사용가능 통해 자유 공학 기본 장단점 채용 실무 경영진 경우 3개월 수습 수습기간 적인 리스 꿈 우리 초기 스타트 업 개발실장 실장 개발본부장 본부장 사랑 일 원 우대사항 사항 접목 가능자 런 칭 3년 사업 직무 발생 때 법 모색 처음 끝 1인 출시 경제력 기존 저하 으 항상 고심 표준 반응 진행 웨어 메타 지속적 주변 대화 객체 2 3 거시 구성 활용 관리 시스템 사용 관련 타 언어 프레임 워크 전자 서명 설정 분야 생성 대회 입상 전공 석 소지 국제 안 중인 제공 강화 컴퓨터 과학 기반 최신 재현 데 수 보신 도구 다양 기술 적용 공유 적극 참여 아 키 텍 베이스 가능 개선 보유 이슈 사용경험 망 공급 물류 이해도 이용 통합 인증 머신 러닝 연동 구현 아래 과 라 서울 강남 강남구 테헤란 7 7길 길 12 허 허바허바 바 질문 무엇 백 보유자 즐거움 새  의견 재생 전기 누구 세상 지배 디드 무선 충전 빌딩 3층 튜 링 층 말 역 역삼 역삼동 삼동 문화 핵심 가치 감당 한마디 외면 감사 제각 해보신 인터 렉 접근 근무 근무조건 조건 근무형태 형태 정규직 개월 사용 사용기간 기간 후 전환 여부 티브 욕심 사고 서류 최종 결과 결과안내 안내 1주 주 2주 주 소요 최종합격 합격 불합격 서류전형 전형 인터뷰전형 서류접수 사전 사전과제 과제 인터뷰 1 1차 차 2 2차 우대 자신 방식 트러블 슈팅 강점 가지신 5 5년 년 이상 외 수행 스크 래핑 선언 레 어디 욕 의지 검색 매력적 풀 을 단위 경험자 고려 사용자 까에 대 고민 리싱 싱 사람 평소 생각 흥미 저희 심의 예시 회사명 회원권 일괄 행태 최종면접 실부면접 상시 공고 공고기간 상시채용 통신사 부서와의 조율 라벨 고용행태 어드 성장프로젝트 전공자프로젝트 내고 마지막 그간 게시 게시일 회사 사용법 변수 명과 변수명과 각각 구성요소 생활 믹스 계획 영사 24 점 긴장 건축 프롬 제기 판단 분류 긴장 기억 하나 조직 현실 내가 지금 무언가 하루하루 전국 내 스택 중심 이력서 첨부 서류 검토 도움 상용 오픈 소스 기여 기여해본 해본 딥  '''
# word_tokens_1 = kkma.nouns(ll)

# result_1 = [word_1 for word_1 in word_tokens_1 if not word_1 in stop_words_1]

# print('불용어 제거 전 :',word_tokens_1) 
# print('불용어 제거 후 :',result_1)

In [12]:
data['이용하는기술스택/우대사항'] = data['이용하는기술스택/우대사항'].fillna("1")

In [13]:
data.loc[data['이용하는기술스택/우대사항'] == 1] = "인트아니다"

In [14]:
result_c = []

for aaa in data['이용하는기술스택/우대사항']:
    if type(aaa) == int:
        print(aaa)
    mm = ''.join(aaa.replace('•',' ').replace('ㆍ',' ').replace('등',' ').replace('가능한자', ' ').replace('하신분', ' ').replace('해본경험', ' ').replace('해보신분',' ').replace('+\d만원',' ').replace('+\d억',' '))
    stop_words = '''이해 분 관심 웹 27001 보안과 규정 스타트업소통 불편 자격요건 수검 숙력 집단 협업 대치 대치동 라즈베리 센서 열의선정 선정 미래 10명 서치콘솔 강한 면제 군면제 프로세스 웹성능 의무 남성 응용경험 밑바닥 조절 장애대응 프리세일즈 예방 재개발 협력사 검색서비스 기업데이터 엑 엑셈 어필 스타일링 메시징 해당직무 데이트 난이도 에너지골프 선정 검색엔진최적화 여행시스템웹 적극성단위테스트 노하우측정 기여경험여 토큰화 오아시스 바리 멤버십 업무해본 2시간 튜드 실행력 기능테스트 개발자자발적 100만원 종단테스트 종단 토큰증권 앰플 공공 리액트로 국가사업 맨 마인드개발자 해치랩스의 카메라 업그레이드 토큰 팀빌딩 리딩해본 유사언어 폴리곤 쇼핑물 랩스 피칭 업그레이더블 데스크톱 데스크톱앱 출력 학습라이브 인터랙션 성실 워크로드 지속적인 성능향상 측면 공공기관 지식구축 언어권 베스트 거리낌 적성 중급 방어 방어기술 노하우플랫폼 스타트업의 기여경험 사이버 프론트성능 공공데이터 개방 프로젝트전문 논술 노드 포트폴리오대용량 마음운용 활용경험 설계자 가격 원칙 준수자 음악유통정산 유통 참여경험 차키 축구 테크성능 인하우스 계정 학술지 솔루션인공지능 벡터 개발이력 복리후생비머스 자사 자사상품 커뮤니케이션배포 만난 반대 기한 토끼굴 배려심 외주 예약서비스 하우스 토대 록체인 지향점 음악 레벨 협업스킬 통용 비기능 먼트 간부블록 샷 애착 가상자산 자산 데일리 차원 이중화 파인 문서관리시스템 매사 계발 효율성 검증 아시 동기부여 업무영역 한계 팬덤 성장과정 학습프로젝트 적합성 정확 호텔예약엔진 컨벤션 이해도과 게임서버 소프트스킬알고리즘 성능최적화 브랜치 발생시 에이전시 부하테스트 싸이클 소규모 한정적 여행시스템데이터 열의 지식애니메이션 기능단위 서버관리 화의 겅 데이터기반 열의배포 커스텀 자유형식 성장유관 팩터링 물류시스템 정산 키택 자기개발상용서비스 독립적 롱위켄드 태스크 애니메이션 합격자 공학계열 업무용 리조트 유지보수성 서치 자발적 뮤케이션 고도화최적화 스크럼 실서버 백오피스 팀웍블록 망설임 생태계 항공예약엔진 호텔 예약 주니어 탐구심 줄 전시전자공학 사이드 고객중심 유니티3 단위테스트 커뮤니케이션데이터 디자인시스템 원리 템플 릿 필요성 점진적 웹앱 분산관리 라즈베리파이 사용경험자 2호선 개발자이지만 이지만 웹소설 자극 소설 허들 운용 심리적 신뢰성검사 선릉역 적재 물리적 수집 기본지 프런트 신뢰성 비개발조직 리더쉽 개발경력플랫폼 개발자서버 무중단 인근 과의 대주주 횡포 배포관리 결격사항 지식배포 디자인팀 주식 웹아키텍처 운영관련 포트포워딩 워딩 100만건 코드버전 발표 인트라넷 커뮤니케이션광고 타고 광고플랫폼 고도몰 영카트 카트 정보통신공학 데이타고 커뮤니케이션협업 기초블록 환경자 대인관계 의사결정 콘솔 활동적 관련전공자 굿스플 노하우 데이터링 스포츠 원인 교육기관 희망 희망자 이수 제자 양성보안 실력 프론트모듈 흐름제어 수상 비개발조직 주도적쇼핑물 레플리카 비법 세일 즈맵 마음앱 시니어 편입 전문연구요원 세일즈맵 코드품질 융합 21 골프 안주 운영환경 관장 에이 속도 분산처리 헤더 고성능 대량 캐시 대치 기술조직 개발조직 웹표준  대치동 무선충전 이전 전문적 정부 툰 이슈관리 소셜카지노 유관업무 툰 지사 부천지사 부천 거부함 기술도입 개발문화 작용 상호작용 자일 이공 전공자이공 기사 협업역량 성장커뮤니케이션 선릉역 소프트스킬상태관리 세서 점심 구비 도전정신 컨텐츠 시나리오 가설 아이폰 이타 동료스토어 능력플리케이 국가유공자 고용촉진지원금 촉진 보신분 처리해본 지침 개인정보보호 인증업무 번역 유관분야 셈 일치 전시솔루션 친화력카지노 장기간 테스트케이스 기구 코드베이스 질량 통제 제약사 추출 게임업계 부탁드립 드립 해결책자동화 보드게임 정립 품질관리 전업 사교적 겸험이 험이 행동로그 스토리 유저향 내부통제 10인 바이오업계 기술특허 수강 구조화 협업도구 전시역략 프로젝트신약 질량분석 연구개발  헬스케어 전자서명 흐름도 주도성 내외부 플라이 스에 점검 일어 일로 계약 영향평가 입사확정 순서 입사일로 일로 계약 무역량 실무역량 객체지향 의욕 멘토링 획득 재무 -24 시트 스타일시트 양성패턴 유경험자패턴 능력콘텐츠 이론콘텐츠 물리 스토리텔링 간결함 무리 밖 회피 완벽 상화 계좌 금융기관 감지 우신분 지라 주일 1주일 전과정 공동의 비스니스 오브젝트 반응형 학부 페어 출판 피드백 컴퓨팅 세팅 퍼블릭 해결책 시용기간 웹사이트 서빙 알고리즘 데이터구조 외국어 크루 교육경험 라이브러리 웹서비스 중국어 개발환경 앱유닛 디자인 웹최적화 사용기간  학습공급망 물류산업 브로커 대응 부동산 부동산시장 콘텐츠 석박사 컴퓨터과학 인프라 관련경험 형상관리 군전역 전역 갈망 신규서비스 스페이스 국어 구현경험 플레이어 혁신적 공부 듀테크 부채 밍 주도적쇼핑몰 간부 간부런칭 협력 협력툴 정책 앱영어 성격 상호 배려 그림 헌신 거래소 증권사 물류서비스 연동경험 지그재그 사방 오토 생산성 적응력 개발문서 문서 집계 개별 보훈 보훈대상자 복구 학력 미터 미터함수 동시성 크로스플랫폼 주도적인 과제전형 운영경험 32 두이 두이노 고도화 플러그인 지식온라인 생산 생산적인 어려움 퀄리티 주인의식 프리 향상 3.0 외부 보고 욕구 기록 서로 커뮤니티 심층 제시 가치관 이후 개별적 완료 전산학 인덱스 백업 프리미티브 체인긍정적 호환성 크로스 브라우징 터보 레포 지식원격 원격 인터페이스 케이스 파트너 트래 백로 진행과정 운영플랫폼 적합 전적 유닛테스트 자세 도전적 나열 장점 리더십 팔로 팔로워십 밸런스 소스분석 앤드 에너지복잡 복잡 데이터셋 비정형 메인 메인테이너 이너 2인 암호학적 학적 클린코드 파일링 퍼포먼스 이벤트 동향 리소스 확장성 기초 레이션 스트레이션 능통자 통자 입사지원서 마음 집중 기술적 부분 돌파 패키지 국제화 팀단위 모빌 빌리티 처리개념 타인 컴포넌트 비디오 배치 젠 킨스 액션 토스타입 손코딩 정보기술 팀간의 헥사고날 모듈 시각화 텍스트 탐지 부하 스토어 채팅 영상통화 통화 스터디 성능개선 구체적 토스플레이스 개발자실시간 모니터링 구축경험 함수 전문가 공개적 제품팀 린스 공학과 블랙박스 로그 유튜브솔루션 보안관제 디버그 국제학술대회 예측 파악 하이브리드 자기주도적 파이프라인 타이핑 스케일 스케일업 압축 화질측정지표 유지보수 직관적 실용적 내시 성공적 테스트코드 기술스택 인증심사 심사 보안장비 스크립트 소셜 시야 학회 저자 리눅스학습 인증심사원 대용량서비스 영상처리 연구활동 브라우저 협업툴 멀티미디어 동영상압축포맷 포맷 주관적 개관 화질 사이언스 문서화 팀워크 월드 상거래 비지니스 인터넷 인터넷사업 정보보호 법률적 전력 프로세싱 플리케이션 의료분야 저널 경량 샤시 운전가능 시뮬레이션 자차 박사과정구글 페이스북 제품군 품질향상 셋팅 알람 사이클 회화 문제해결역량 트레이서 열정적 강연 강의 교수 백신 드라이버 치트 제어 개발사 경진대회 다종 박사학위 박사과정패턴 라이브 아바 커스 커스터마이징 코딩 코딩테스트 신중 방법론 비관계형 경력자 지식석사 석사학위 영어회화 병렬 트랙 레코드 지리정보체계 이해관계자 관계자 오너쉽 오너십 기획자 스프린트 프로그램 바이오 동종업계 표준화 확장 유니티 제어기 차량연동 그래픽스 자율주행 능통 체계적 동영상 전력시장 정보통신 수학 엔진 저장소 학사 근거리 근거리무선통신 시계열 영상 팀웍 인더스트리 석사 테이블 오픈마켓 마켓 주문 중개 개선점 선점 엔드유저 개선안 아이디어 논리적 분산 신용평가모델 임상 기계 기계공학 전기관련 자격증 주요서비스 유경험자 튜닝 리포팅 비동기 병역 병역특례 특례 보충역 취약점 동아리 해킹대회 카카오 테크이미지 동료 시너지 엔지니어링 의사소통 수평적 자기개발 자료구조 운영체제 아이 프로그래밍 가상 리딩 티스레드 마인드셋 네이버 신발 패션 의류 업무방식 창의적인 토론 자동화해본 마케팅 리터 키오스 학습 주도적 전산 계열 주도적데이터 도메인 프로덕트 해결법 깃 형식 동기 스펙 코드리뷰 허브 론칭 기술기반 규격 공인 메모리 전화 공인인증 마이크로 대기업 졸업자 팀원 지도플랫폼 금융데이터 모델링 재생에너지 문제점 향상 커뮤니케이션 리더 마인드  멀티 쓰레드 서버 개발경험 모델 영문 논문 분석 주세 이유 요 토끼 굴 피그 교체 업소 개발자문제해결 아가 유행 해소 스타으업의 3차 적극성 시간동안 해치 지갑 출하신 업의 호출 사용중인 대시 고민해본 보여 보여주실 명령어 보드 소켓 열린 재충전 외국 2년이상 확산 고급 분리 액 트로 변경해본 기여겸엄혀 멘 설명서 개발주기 식별 고급사용 가이드라인 채용전형 1차면접 인성검사 2차면접 실천 혹 곤 비트 럼 그레이 더블 응답 시간투자 계신 노고 로드 드리고 안내드리고 분의 심도 소정의 후보자분의 미션 스의 역할수행 기간동안 추하신 3시간 대해 1차면접 채용전형 인성검사 결함 오류 버그 주시 책임 기념품 안정적 책임 주도적 포상휴가 취미 책임감 능동적 긍정적 최적화 사용기간사내동호회 가족 생일날 증진 부정적 심리 상품권 반차 사외 직무교육 확대 어학 편리 어린이집 휴향시설 운동비 재발 재발방지대책 방지 대책 수립 적극적인센티브 복리후생비대 성장디버깅 윈도 측정치 구축 배포 마우스 대외 대외서비스 간편 간편결제 현업 탐색 한편 사전질물 입사안내 네이션 운항 입상자 입상자자동차 성장 금융 미림 미림타워 지역 대형 4주 메신저 이기 기종 흉 지식보유 노환경 과장급 악석 경력3년이상 여자 영어가능자 입상자자동차 입상자 운항 입사안내 출장가능자 출장 이중 복수 유튜브프로메 테우스 우스 근무지역 경력직 채용유형 인원 채용인원 채용정보 파일형태 기타사항 미스 복수 복수전공 훈련비클라우드 이중전공 정의 적극적 운영중인 해결 비대칭 제작 혁신 기획 지식실시간 지도 중계 프로토 지식 온라인 광고 업자 클린 처리 항상 런칭 효율적 근본적 호기심  프론트 엔드 전문 다양성 추진력 글로벌 테크 디지털  차량 탐구 성능 사용기간 구성원 조직문화 핵심가치 진보 변화 개념적 도전 목표 업계 신기술 완성도 블록 체인 데이터 소프트 수정 메시지 품질 노력 서류검토 포트폴리오 역량 능력 보완 개선의지 사용중 숙달 성찰 바꾸기 채용절차 환경내 경내 듀 진단 내외 오프라인 스트레스 평판 평판조회 조회 결산 텔 최종합류 처우협의 기술과제 전달 주차별 차별 지연 직접 캡 연계해본 경험해본 8년 19 시차 4대보험 주5일 릉 랏 방문 정비 스플 적적 긍적적 기기 긍 코드 헬스 제로 빌드 5호선 화곡역 5분 10분이내 럭 난독 하위 호환 100만 폰 5일제 일제 17 강서 강서구 화곡동 건전 1회 사내 저녁 후생비 저녁식대 야간 생일 결혼기념일 분당 스마트 변경 시장 27701 유기 5인 실시 활성화 점수 성남 성남시 분당 촌 연세대 연세대학교 2곳 곳 분당 분당구 판교 723 404 404호 서대문 서대문구 연세 광 광혜관 라온 메디 스테이지 차피 4918 회수 의의 동반 축하 부모 경조사비 조사비 자율복장 복장 회사콘도 콘도 육아 워크샵 샵 격년 테니스 조사용품 용품 포상 10년 포상금 여름 부서별 회식비 음료 각자 개개인 주인 굿 준수 좋아 7년 무엇인가 인가 콘 솔 다루 커피챗 브렌 전략 장기근속 근속 장기 서울시 사용기간 소유자 보유자 2가지 스타일 팬 덤 분실무 견고 분우대사항 달성 지속가능 기법 존중 결정 근무환경 포구 중간 활동  중간관리자 관리자 직장 요원 더쉽 가지신분 근거주자 주자 장기근무가능자 대인 직업 훈련 기관 직업훈련기관 효과적인 인재 인재영입 영입 매니저와의 와의 주의 주의사항 동일 복리 복리후생 후생 일반적 운동 행복 행복하자 하자 몸 두시 기업내 커리어 여의대 108 파크 파크원 타워1 38 38층 상위 연봉 오전 11 11시 생략 연결 쇼핑 퇴직금 연차 연차휴가 모성 모성보호휴가 경조사 경조휴가 경조금 조금 화환 효과 피해자 오신 최선의 선의 대안 지인 시일 담당자 연락 방안 컨트리 컨트리뷰 무관 비용절감 절감 부담 휴양 핀 브라 우징 부탁 방법 위주 수행업무 개인홈페이지 홈페이지 1시간 정식 암호 화폐 어린이 슬픔 화곡 기쁨 장례 지인분 1년 여정 비어 90 90분 스킬 인상 셰어 라운드 6개월 있음 최소화 주민 주민번호 번호 개인정보 삭제 요함 중단 어제 오늘 이것 선배 설명 표현 안정성 삶 몰입 종속 일정 산정 2회 포지션 편안 출근버스 대중교통 운행 주택 주택자금대출 자금 통근 공간 단체보험제도 3인이상 경청 보완 사기 열정 포토 파이어 확정 주기 즉시 용인 원활 동작 용인시 수지구 09 상여금 업무우수자 기혼 기혼직원 식사 청년 내일 음공제 공제 18일 미사용시 사용시 수당 상금 임직원 건강검진비 보조 소개서 옵션 톤 해커 참여해본 전직 모두 현역 불가 본인만 바른 예정경력기술서 기술서 필요시 셜 3인 습관화 현지 메시 회복 의료비 단체보험 차이점 지가 임원진 대신 과제테스트 식비 10만 상당 건강비 교육비 정기 리프 리프레쉬 쉬 독서 독서모임 모임 시즌 멤버쉽단체 고객사 조화 주차 이자지원제도 이자 주거 대출 유의사항 최종결과 전월세 모집 조기 동의 허위사실 비지능 구매 부족 실적 모노 회의 공감 답변 명령 미들 설치 숙박 학점 학점은행 은행 성취감 선호해 호해 벡 문화적 검사 추가 유연 유연근무제도 제도 셋째 셋째주 금요일 1시 넷째 넷째주 월요일 출근 롱 롱위켄트 켄드 야근 식대 심야 교육지원 2년시 채널 튜브 관제 대한 계속 랍격자 수시 마감 3년이상 365 보수성 친숙 복지비 휴가비 통신비 종합건강검진 이해심 모범 애정 순위 엣 귀하 사회 이바지 라도 분1 만들기 13버전 엘 스틱 리초트 급여보장 초과 링크 뮤 우수자 버전 중앙 습득 참가 학술 게재 파이프 준비 요구사항 단계 추천해 공통 해당 국가 유공자 장애인 취업 취업보호대상자 대상자 관계법령 법령 7일 며칠 헥 날 시작 혜택 개선해본 현재 이내 이메일 간의 리팩 토스 플레이스 함 창업 처우조건 포스 타입 내규 1일 8시간 수습조건 근무장소 장소 4길 강남역 1번 출구 2분 지원시 참고 참고사항 모든 권장 성공 사례 포함 현장 손 소정 참여비 간소화 4년 성과 블랙 박스 자격 취득자 투브 이외 라우드 워크플 넷 신규 체계화 장려 석식 신청 커피 무제한 동급 장비 제품 퇴근 교통비 연말 겨울 방학 별도 승인 워라 보장 재택근무 비용 도서 구입 수업 환경 케어 중시 펀 마크 비전 중요성 신분 출퇴근 근무시간 조정 동시 해외여행 결격 의사 관적 유사 유지 보수 지면 264 탑재 보호 사원 우수 기본적인 지급 결격사유 시용 사유 운전 글 페이스 북 4인 전체 접속 2,000 2,000명 수용 뉴 럴 자율 주행 지리 체계 카페테리 테리 건강 검진 경조 수원 위치 고용 고용형태 검출 추적 86 64 버싱 전형절차 면접 제출서류 사지원서 지원서 양식 명절 선물 동호회 종합검진 독감 독감주사 마이 제외 모습 부여 센터 센터필드 필드 타워 교육장 경기 경기도 우대요건 요건 규모 소지자  13482주사 국내 출산 자녀 입학 포 비즈 니스 요청사항 후속 영업 끈기 관점 소유 도입 필요 경진 검수 반영 인식 지원자 안식 휴가 지원금 6년 9 9년 근속 현금 복지 포인트 편의 시설 녀 수면 수면실 간식 휴식 카페 성향 추구 결과물 연구팀장 팀장 인력 분 지향 작성 버스 재직 근무요일 체제 부서 우선 정리 읽기 쓰기 구상 독성 유니 영역 의식 전장 26262 특성 규칙 중점적 지표 유의미 고가 용성 졸업 수자 역할 번 관련교육 2년 신용 장애 산업과 수자1 4 분관련학과졸업 상응 동종 전사 유의 리드해본 구분 화면 오너 십 본인 파트 지능 발전 편집 욕망 거리 기타 크립 서비스여 혼자 집 벨 벨루가 루가 큐칙 주류 산없과 기능 분과 대상 인공 제안구애 주요 작업 시험 일부 최소 3개 1개 플 케이 인더스 트리 요구 운 영해 운영해 가능자1 너리 취약 뿐 중요시 빅 상태 비 실행 반복 레드 요청 일상 파 파티 응용 셋 진심 기업 실제 대체 분위기 자기주 다량 만족 중요 여기 실패 창의 방향 구조적 활용해 용해 각종 주도 자기 사이트 종사 수준 맥락 통합적 설계해본 요일 극복 후보자 치열 논의 수준의 커버 리지 다수 유저 오스 결제 플랫 펌 드로 이드 추천 다각 다각도로 도로 오피스 총괄 참석 전반적 핏 내부 상황 변동 내용 스스로 설득 시간 체크 합류 종합적 영향 필수 주소 선택 형시 1로 로 최대 등기 확인 앞 기대 실무진 대면 60 60분 가량 중심적 이야기 자리 시도 5일 일 40 40시간 7시 30 30분-10시 -10 30분 처우 보상 개인 바탕 협의해 절차 입사 입사지원 지원 재직해본 형성 누비 랩 개발팀 8 다음 션 근무지 서울특별 서울특별시 625 덕명 덕명빌딩 6층 삼성 삼성역 도보 3분  최고 장벽 기준 사업부 자체 프로 동안 급여 100 지금 평가 연장 종료 과정 제출 증빙 증빙자료 자료 사실 허위 기재 발견 취소 콜 8 론 재미 직군 사용가능 통해 자유 공학 기본 장단점 채용 실무 경영진 경우 3개월 수습 수습기간 적인 리스 꿈 우리 초기 스타트 업 개발실장 실장 개발본부장 본부장 사랑 일 원 우대사항 사항 접목 가능자 런 칭 3년 사업 직무 발생 때 법 모색 처음 끝 1인 출시 경제력 기존 저하 으 항상 고심 표준 반응 진행 웨어 메타 지속적 주변 대화 객체 2 3 거시 구성 활용 관리 시스템 사용 관련 타 언어 프레임 워크 전자 서명 설정 분야 생성 대회 입상 전공 석 소지 국제 안 중인 제공 강화 컴퓨터 과학 기반 최신 재현 데 수 보신 도구 다양 기술 적용 공유 적극 참여 아 키 텍 베이스 가능 개선 보유 이슈 사용경험 망 공급 물류 이해도 이용 통합 인증 머신 러닝 연동 구현 아래 과 라 서울 강남 강남구 테헤란 7 7길 길 12 허 허바허바 바 질문 무엇 백 보유자 즐거움 새  의견 재생 전기 누구 세상 지배 디드 무선 충전 빌딩 3층 튜 링 층 말 역 역삼 역삼동 삼동 문화 핵심 가치 감당 한마디 외면 감사 제각 해보신 인터 렉 접근 근무 근무조건 조건 근무형태 형태 정규직 개월 사용 사용기간 기간 후 전환 여부 티브 욕심 사고 서류 최종 결과 결과안내 안내 1주 주 2주 주 소요 최종합격 합격 불합격 서류전형 전형 인터뷰전형 서류접수 사전 사전과제 과제 인터뷰 1 1차 차 2 2차 우대 자신 방식 트러블 슈팅 강점 가지신 5 5년 년 이상 외 수행 스크 래핑 선언 레 어디 욕 의지 검색 매력적 풀 을 단위 경험자 고려 사용자 까에 대 고민 리싱 싱 사람 평소 생각 흥미 저희 심의 예시 회사명 회원권 일괄 행태 최종면접 실부면접 상시 공고 공고기간 상시채용 통신사 부서와의 조율 라벨 고용행태 어드 성장프로젝트 전공자프로젝트 내고 마지막 그간 게시 게시일 회사 사용법 변수 명과 변수명과 각각 구성요소 생활 믹스 계획 영사 24 점 긴장 건축 프롬 제기 판단 분류 긴장 기억 하나 조직 현실 내가 지금 무언가 하루하루 전국 내 스택 중심 이력서 첨부 서류 검토 도움 상용 오픈 소스 기여 기여해본 해본 딥  '''
    word_tokens = kkma.nouns(mm)
    word_temp_1 = [word for word in word_tokens if word not in stop_words]
    result_c.append(', '.join(word_temp_1))

data['a'] = result_c

result_d = []

for sss in data['이용하는기술스택/우대사항']:
    word_temp_2 = re.findall(r'[a-zA-Z\s]+', sss)
    result_d.append(', '.join(word_temp_2))

data['b'] = result_d

data['c'] = data['a'] + ', ' + data['b']

data.drop(columns=['a','b'], inplace = True)
data.drop(columns=['이용하는기술스택/우대사항'], inplace = True)
data.rename(columns={'c':'이용하는기술스택/우대사항'}, inplace = True)

In [15]:
data['이용하는기술스택/우대사항'] = data['이용하는기술스택/우대사항'].apply(lambda x: ','.join(word.strip() for word in x.split(',') if word.strip()))

In [16]:
data.자격요건 = data.자격요건.fillna('1')

In [17]:
data['자격요건'] = data['자격요건'].fillna(1)

lll = ''
for eee in data['자격요건']:
    if eee != 1:
        lll += ''.join(eee.replace('경력\d년', ', ').replace('최소\d', ', ').replace('개발', ', ').replace('구축', ', ').replace('경력', ', ').replace('기반', ', ').replace('성능', ', ').replace('경험', ', ').replace('능력', ', ').replace('•',', ').replace('ㆍ',', ').replace('등',', ').replace('가능한자', ', ').replace('하신분', ', ').replace('해본경험', ', ').replace('해보신분',', ').replace('+\d만원',', ').replace('+\d억',', '))
    else:
        pass


stop_words_2 = ''' 하나 분 년 개 1 이티브앱프로젝트 11 티브앱 분석 러스 오픈소스 수강자유관업무3 코드관리 상급 백엔드개발5 상급이상 기술학습 초기설정 설정 완성 전체과정 웹애플리케이션개발 원칙 서버개발자 리스 디자인직군 생각하백 대시보드생성 하백엔드엔지니어경력 연동가능 합리적인의사결정 대시 추진 프리랜서 유관분야대학교 학사졸업자 결격사유프론트 하이브리드 하이브리드웹개발가능 프론트개발경험5 있으신분 맞추기 싱크 팀멤버 최소4년이상 신입지원불가 프론프엔드경력3 자기주도적업무 유지 개념 주요개념 유지보수경험 보수 리딩 리딩경험 중1개언어 경력3년이상 관리운영경험 무중단서비스 주기 스키마설계 소프트웨어생명주기 생명 기반환경 8년이상 중단 트래픽 금융상품 서비스경험 상품 책임감 투자 주도적 경력보유 시니어 구현능력보유 데이터처리 서버개발경험보유 최적화경험 이해보유 클라우드서비스관리경험 통신 개발경험보유 통신서버개발경험 개발자우대 라이브러리 플랫폼 최적화경험보유 구성요소개발경험 기술전문가 최적화 전문가 성능 프론트엔드개발경력 고객요구사항 제품개발경험보유 구성경험보유 최소5년이상 코칭 플랫폼구축 양성 개발언어 대졸이상학력 전과제 도전 개발이력10년 편집기 경력5년이상 팔로워 개발효율 사용경험보유 워크활용 동료 활용개발환경구성 중급이상실력 개발경력3년 프로젝트개발경험 애플리케이션 디자인시스템 정규직 6개월시용기간 정규직전환여부검토 전산 전산개발경력최소 숙련자 전환 프로그래밍언어중급이상 중급 논리적 대화 티스 5년개발경험 운영업무관련역량 병역 병역면제 면제 업무이해 체계 조합 문서화가 이해도 개발경력4년 체계적 소스기술 업무관련 업무경력5년 관리능력 매너 친절 외국어 능통 해외출장 출장 통합경험 결격사유 서비스개발경험 직접코딩 사진 마이크로 습관 기술적 기술적지식 기술적시야 시야 호기심 서비스설계 개발능력 테스트설계 수행능력 깃 소통능력 프로그래밍언어 개발툴사용능력 논리적인사고 기획자 팀원간 기반웹서비스개발 개발경험자 개발실무경력3 언어모델 관리경험3년 방법론 사전학습 효율적 모델 파라미터 지식보유 튜닝 결합 모니터링경험보유 공부 기초지식 서버개발경험 데이터베이스설계 관리경험 유관경력 웹지식 논리 백엔드개발경력 의사결정역량 인프라 인프라관리경험3 운영경험3년 형상관리도구사용 서비스운영경험3 배포경험 프로그래밍언어 열정 성실 전공무관 해석 우선순위 동시 검증 전략적 촉진자 전략적사고 사고 역량보유자 동향 업적 획기적 관계형성 관련전공자 공무 개발경력2년 대용량 데이터 데이터 데이터 서비스개발 획득 엔지니어 디자인 괄목 성과 내신 리터 러시 수행업무 리더십역할경험 성과 4년대졸자 10년이상프로덕트 리더십 관리 열정 성실 성실전공무관 공무 개발경력2년 대용량데이터 데이터 최신기술 트렌드 디버깅스킬 대처 협력 팀워크 싸이클경험 프로젝트 분자격요건 단백 최신트렌드 룰수 부트 기숙사1년 사택 관적 최대5,000만원 27001 진심인분 취소 유니 있으신분웹퍼블리싱 급경험 여진 때적극적 포기 중간 타시스템과의 분필수자격요건 시멘 틱 지지 기반백 얼 분최소2년 라도 경우채용 중하나라 나라 모든 스플 집착 을경험 1년포함 모집 모집인원 위의조건 개발자로 자로 진심 개발신입 운개발환경 한협업 개발시 호출 3년웹 와이어 랫 폼 동의 액 회원 티비 스킬보유자 부서와의 전반 분수 맥락 징경력 분1년 공감대 안건 고유 누락 부문 설과 실질 전반적 전적 마케 기획이외 면접후결정 자간 소지자 그중 그중1년이상 중2가지 직무 설과인사이트 자타 현상태 기본적인영어문 해력 학분야 팀과의 신약 문제해결능력3 화한 보증금 일 변환 단기 수 있음 수있음 을2년간 분필수요건 지속적인 경험필요 인력 분이벤트로그설계 분개발자 원어 채용프로세스서류전형 2차면접 8년정도 결제 자격사항 지식1 기술과툴을사용해 작성가능6 경우면접 비지급 지급 대중교통 대중교통이동 이동 시외버스 기차 영수증 중 증빙시지원가능 빙시 연고지 기숙사 서의개발경험 졸업예정자 도포함 능력중 마인드모바일 문야 기기 기술서 경력기술서 외지역 력를 얼 아래3가지질문 답변포함 의안 수있음 있음 긍 의안정성 수조건 사업 센터 게신분 이상경력필요 연한 편입 경험자관련경력2 보유자임 졸업증 가지신분백 개발가능4 상황 이상4년이상 사용법 만큼 소양 52 지인 사용자대상 대상 분실무5년 위의 창성 아래의 편입신청 선착순 활용경력 컨셉 편입인원 타인 다회경 주요업무 희망연봉수준기재 사유 가지신분위 구성원 탈 누구 지환 팀원과의 페르 소나 이상경험자 때어려움 하면서변화 지환영 특별 공개 하이신분노션 분사용 모니터링하면서장애 자격제한 하면서 파이 아이디 면접시 해당과제 진행여부 과제진행 1차면접사전 내외부 위한아 분1차면접 관점 파악 한아 그룹 그룹사 자산구매 배포판무관 판무관 육아 의도 심층 급변 년이상실무경험자 휴직 대체 대체사용 구매 분실무경력2 부산 분필수경험 다양한도구들 한도 3년이상제품 성별 의학사이 최소한가지이상 인의 견 '최소1년이상 네트 하이 션에 상위 3년이상서비스 제외 한번 도움 제출 이유 바른 요원가능3년 주제 2회 주제무관 2회이상 접시 손끝 접시준비서류 과목 기본과목 준비 업종 트업 트업문화적응력 엔드개발경험 실무백 필요역량스택 이상백 게이 룰 창업멤버 시대 스타트업5,000억 7년내 멤버 3개이상 기반실무개발경력 기본적인이해 7급 명 이상유경험자 리드 800급 800 중최소1개 구조대신 대신 분기술스택 밍구현능력 제이 32 3개 기초적인프로 중가능 언어1개이상 언어중 내의 10년내의 기반하 반하 두려움 분전자 재현 깊이 의식 중이 수원 지역 이상12년이하 다음기술 6.0 무리 이상우대사항 외하나 당황 컴퓨터공학과학사이 개발자체 믿음 정신과 대우 배치 끝 과합리적 운영경험최소3 공통 공통기능 용성 업계최고대우 공학과 이력서상희망연봉 킹 마이 희망 기재필수학 1.8이상 관련업무경력5 자격요건 분성별무관 년이상기술스택 키오스 가능자7 환경하 1.8 경하 보유자5 활용가능자6 자백 분만4년이상 가지신분만8년 있으 있으신분 참여를 시분 학습의지5년 계신분 미들 안드로 실제최소3명 년이상경험 폴리오 분경력3년 미만 부심 필수사전질문 트웰브 년이상경력자관련 채용프로세스 근무장소 장소 채용절차 이상서비스 과의사소통 경우관련학과학 계속 기존 중이신분 활용가능자4 제로 가지신분통사항 분통 아시 보람 디버 흥미 초기 담장자 때담당자 구멍 일의 일의구멍 지금 단시간 내몰입 조직내 사이 2주단위 당업무 분최소4년 내재 분금융 개발충 간원활 자사 의협업경험 기개발 회사성공 숙지 초기스타트 마련 선아래 기대 분로그 경험2 렌치 베 디 부담 모두 기구 최소하나 기구축 모두가능 면접접차 직군과의 과의원활 접근 구두 소지 지속 셰 신분3 처리 이상가능 후배 분경력5년 7인이상규모 케이션역량 케이션설계 스터링 위경험 트개 721 전체 반응 분최소5년 코드적극적 의향 스를구 업환경이익숙하 실무진 진행 진행가능 1차인터뷰이후 010 5596 0870 승 대표 1콜챗 연락 연락주세요 세요 우리팀 지원이전 지원시지원서 시지 직전연봉 유의미 명시 확신 언제  최종합격순 실무인터뷰 아래질문 2차대면면접 우신분 최신스펙에대 욕심 분7년이상 힘 욕구 기타언어하나 이상7년이하 플리케이션 마음씨 페이지 주세 선호 구현경험 구조 팀인터뷰 자유양식 파일권장 혼자 공인 결과물 중심 본인만 외조직 컨트 를연결 인원 업자 고용 고용형태 급여변동 변동 서울시 서울시용산구이태원로 서류전형 1차기술과제 기술인터뷰 인터뷰 3.1 맵수 주2 3회 원리 업무수행가능자 2015 항목 취득 협의후결정학력 년이상자 이상자 항목중 이력서내 연봉필수기재 주5일 금 시차 근무일시 근무형태 근무조건 중최소한 분03 03 90 90일내발급 발급 서가 분04 04 05 이해4 이풍 머 풀이가능 1억원3 도환영 환영 롤 2기반 젠 간의 내가 12 유경험자 과적 방법 방식 24시간 미니 형태 확인 3기본지식 배경 혼합 외언어 부족 말 캐치 충족 충족가능 총백 요원 요건 년이상경력 어가 핀 셋 체감 피력 상대 고려 일정 의가 독성 기반웹 무엇인가 인가 즐거움분초 이상5년이하 가치 분우대 팀내외 내외 분풀 고성 갈구고성 내고 조건 분초대졸이상 관계 이공 분대졸 과적극적 53 실무5년차이 극적 팀내 자도구 도적 부서 타부서 컴퓨터 신분2 구분 지향 12년 내가작성 차장2 차이상의 3가능 자일 해당공고 기업내 2024 2024년2월 노 하우 처음 원의 원의성장 카 2022 2022년2월 한테스트 성격 전직 시유 대한 텍쳐에 수준 공용 과도 메인 글 각각 종류 통해 한어 필요하다 이번 15년이상 13 간의 대면 칭 채용 타파트 가실 5.0 십 가능자 셀 사양산 가실분 및브 릿 제한 입자 분경력8년 신청 8.1 보유필수 기타 웍 개발4 분지원가능 2017이후 2017 스펙에 저희 현재 인지 곳 신입기준연봉4,000 과의원할 브라 연봉범위 4000 6000 우징 리작성 에이 전시 10년이상 4,000 후협의 실시간 최소1개이상 신분4 7년이상3 몰 교 338 동702호 702 협의 면접 한시스템 3년이하 연봉3,000만원 중1가지이상 1시간 을이과정 군의 직군의사람 안주 처우 처우협의 입사예정일 예정일 확정 분1 9중하나이상 상태 결격 입장 6이상 급 용산 용산구 이태 원로 4년이상3 27 27길39 39 11 질문답변 3줄정도 줄 보신백 기반개발경력3 시점 지원자분의회사선택 3가지 분의 계신향후 향후 골 무엇 계신지식 정도 지원서 스합류에 무엇 계신부분 부분 우려 답변 2층 층 2차 최종합격 아래필수사 질문 합격 상세경 세경 1차코딩테스트 2명3년 안 드로 이드 제작 도로 태도로 우수 실패 대인 이야기 화면 이외 이내 갈분 애착 시도 16 인사 고와 절차 최대 이후 전형 이후단계 기능 필수적 육성 심사 스크 유저 분하나 중한개이상의사용 상의 그것 후재발방지 서버프로 어에 밍언어에 개인 이상전자 비즈니스요구사항 분3년이상 행복 급여 통합 추구 심도 후 개월 6개월사용기간 검토 여부 때행복 생성 10,000장이상 경우포함 런스문서 과오 언더 실내 264 265 수립 을 아래 달리 득능력 관행 템복잡도의 도의 운툴 중요 적용경험 연동 건강 불가 및운영  거시 와의 하다 문법사용 3사용경험 분신입 중요하다 성년자 을통 고민 고가 기재 지원시 어필 7년차 제학 논의 전공자최소 분관련경력3 저널 체제 치트 용화 소켓 그래밍경력 객체 지도 사람 세대 사람과의 분6년이상 양식 자유 입사 서류 지원서 입사지원서 소개서 코치 필요시 모습 중시 과의 해외여행 바탕 런경험 최소한가지 실제 중인 해본 분야 런스 문서 작성 분7 분3 실제운영중인서비스 \d개이상 2개이상 표준 케이션개발경험 경험보유 시장 순위 분개발경력4 풀이 불확실 예정자 남성 지원 기반개발업무경험 상용 한가지이상 15 15년 운영해 풀이경험 풀이 영해 다수 부설 처 중한가지이상언어 학과 계열 이제공 각종 숙자 졸업 중하나이상 프로토 콜 벡 라인 분시드 한자 보유한자 때문 덱싱 분6 소통하면 하면 오버 지양 적정 자기주 분빅 규격 빅 뷰 작용 상호 징 사항 능숙자 분6 1년이상인분 상응 인분 스타트 동작 키 텍 포함 마크 지경 지경험이 험이  명세 명세서에 서에 내부 외부 분학력 한가지 개발이익숙하 장애발생시 오너쉽이 무관 쉽이 이하 년이하 소프트 웨어 문화 5년이상7 리지 스토 재발 방지 시하 우선시하 움 분웹 겸비 분4년이상 이상근무 근무 사용기초 학력무관경력5 단위 동기 비동기이해 분5년이상 서의 발란 물 14 검색 사랑 분2년이상 분분업 주저 시스템 동종 때발 실무 분기본적인 등다량 다량 보신 유관 연봉5,000만원 분필수언어 요소 95 100 100점 점 레벨이상 스택대응 거부감 번 6 7 업계 연봉 000만원 연공5,000만원 스톡3,000만원 3,000 5,000 8,000 8,000만원 1억원이미지 억 이용해보신분 커뮤니케이션능력 모니터링경험 우선 분원 분원활 과제 데 항상 라면 도입 라면툴 거리낌 본인 최고 팔로 마음가짐 내용 해당내용 설파 전파 관련 컨테이너빌드 하나이상 소유자고객문제해결 의사 프로그래밍 분대 빌드 솔루션구축경험4 소스 이자 소유자 위해고객 적용 위해 4 년대 년대졸자 졸자 이상프로덕트 역할 기본 기쁨 불필요 본문제 워크 등딥러닝 계신 때 공자 80 역량보유 해당 중 중1개이상  습득 등 수 수 자 뮤 용량 이상경험 연봉5 동종업계경력6 우대 향상 실력다양 이해프레임 구성 자체 대외 최소5명이상 신분 티 브앱 시작 이용 보유 스킬 분경력 일 케이 원활 케이션 컴퓨팅환경 자격보유 자신 상상 야만족 최소 1가지이상 사용가능 분최소3년 자격 10,000 10,000 장 본분 프롬 프롬프 직접 값 값0.1 0.1 속 차이 하나하나 사전 허브 런 킨스 런분과 경우군필자 분과 무자 필자 개이상언어의사 분2 필수 필수사용자 사용자 툴 션 엔드개발경력3 2년이상 분백 엔드 형성 촉진 형상 의지소유자 수행 도구 가지신분대용량 그 활용 기술 케이션개발 워크개발 능숙하신분5 년이상보유 화가 초대 화가가능 생태계 운기술 10년 분기 분기초적 초적 해보 간 케이션능력 기초 러 스터 기반 참여 요구 역량 오픈 적인 3년이상 성공지표 지표 만 만5년이상 과장2 싸이클 가지신 등 선도 팀원 가능 종업 축하 사용 프레임 동 플리 보유자 성공 스스로 사용능력 언어 관심 새 직군 서비스'''
word_tokens_2 = kkma.nouns(lll)

result_2 = [word_2 for word_2 in word_tokens_2 if not word_2 in stop_words_2]

print('불용어 제거 전 :',word_tokens_2) 
print('불용어 제거 후 :',result_2)

불용어 제거 전 : ['프로그래밍', '프로그래밍언어', '언어', '열정', '성실', '성실전공무관', '전', '공무', '관', '2', '2년', '년', '대용량', '대용량데이터', '데이터', '서비스', '딥', '딥러닝프레임워크', '러닝', '프레임', '워크', '티', '티브앱', '브앱', '싸이클', '이', '이티브앱프로젝트', '프로젝트', '최신', '최신트렌드', '트렌드', '대처', '디버깅', '디버깅스킬', '스킬', '협력', '팀워크', '최신기술', '기술', '형상', '형상관리도구사용', '관리', '도구', '사용', '배포', '수행', '수행업무', '업무', '오픈', '오픈소스', '소스', '4', '4년대졸자', '년대', '졸자', '관련', '관련전공자', '전공자', '10', '10년이상프로덕트', '년', '이상', '프로덕트', '리더십', '리더십역할', '역할', '성과', '획기적', '관계', '관계형성', '형성', '업적', '동향', '분석', '해석', '우선', '우선순위', '순위', '동시', '검증', '전략적', '촉진', '촉진자', '자', '전략적사고', '사고', '역량', '역량보유자', '보유자', '의사', '의사결정역량', '결정', '유관', '지식', '지식보유', '보유', '클라우드', '서비스운영', '운영', '3', '3년', '인프라', '인프라관리', '러', '러스터', '스터', '컨테이너', '깅', '모니터링', '공부', '기초', '기초지식', '서버', '프레임워크', '데이터베이스', '데이터베이스설계', '설계', '백', '백엔드', '엔드', '웹', '웹지식', '클라우드서비스', '논리', '논리적인사고', '적인', '팀원', '팀원간', '간', '웹서비스', '쿠버', '티스', '형상관리', '실무', '언어모델', '모델', '방법론', '사전', '사전학습', '학습', '효율적', '파라미터', '파

In [18]:
result_a = []
try:
    for qqq in data.자격요건:
        kk = ''.join(eee.replace('경력\d년', ', ').replace('최소\d', ', ').replace('개발', ', ').replace('구축', ', ').replace('경력', ', ').replace('기반', ', ').replace('성능', ', ').replace('경험', ', ').replace('능력', ', ').replace('•',', ').replace('ㆍ',', ').replace('등',', ').replace('가능한자', ', ').replace('하신분', ', ').replace('해본경험', ', ').replace('해보신분',', ').replace('+\d만원',', ').replace('+\d억',', '))
        stop_words = ''' 하나 분 년 개 1 14 있으 거부감 이티브앱프로젝트 11 핸들링 티브앱 분석 러스 오픈소스 수강자유관업무3 코드관리 상급 백엔드개발5 상급이상 기술학습 초기설정 설정 완성 전체과정 웹애플리케이션개발 원칙 서버개발자 리스 디자인직군 생각하백 대시보드생성 하백엔드엔지니어경력 연동가능 합리적인의사결정 대시 추진 프리랜서 유관분야대학교 학사졸업자 결격사유프론트 하이브리드 하이브리드웹개발가능 프론트개발경험5 있으신분 맞추기 싱크 팀멤버 최소4년이상 신입지원불가 프론프엔드경력3 자기주도적업무 유지 개념 주요개념 유지보수경험 보수 리딩 리딩경험 중1개언어 경력3년이상 관리운영경험 무중단서비스 주기 스키마설계 소프트웨어생명주기 생명 기반환경 8년이상 중단 트래픽 금융상품 서비스경험 상품 책임감 투자 주도적 경력보유 시니어 구현능력보유 데이터처리 서버개발경험보유 최적화경험 이해보유 클라우드서비스관리경험 통신 개발경험보유 통신서버개발경험 개발자우대 라이브러리 플랫폼 최적화경험보유 구성요소개발경험 기술전문가 최적화 전문가 성능 프론트엔드개발경력 고객요구사항 제품개발경험보유 구성경험보유 최소5년이상 코칭 플랫폼구축 양성 개발언어 대졸이상학력 전과제 도전 개발이력10년 편집기 경력5년이상 팔로워 개발효율 사용경험보유 워크활용 동료 활용개발환경구성 중급이상실력 개발경력3년 프로젝트개발경험 애플리케이션 디자인시스템 정규직 6개월시용기간 정규직전환여부검토 전산 전산개발경력최소 숙련자 전환 프로그래밍언어중급이상 중급 논리적 대화 티스 5년개발경험 운영업무관련역량 병역 병역면제 면제 업무이해 체계 조합 문서화가 이해도 개발경력4년 체계적 소스기술 업무관련 업무경력5년 관리능력 매너 친절 외국어 능통 해외출장 출장 통합경험 결격사유 서비스개발경험 직접코딩 사진 마이크로 습관 기술적 기술적지식 기술적시야 시야 호기심 서비스설계 개발능력 테스트설계 수행능력 깃 소통능력 프로그래밍언어 개발툴사용능력 논리적인사고 기획자 팀원간 기반웹서비스개발 개발경험자 개발실무경력3 언어모델 관리경험3년 방법론 사전학습 효율적 모델 파라미터 지식보유 튜닝 결합 모니터링경험보유 공부 기초지식 서버개발경험 데이터베이스설계 관리경험 유관경력 웹지식 논리 백엔드개발경력 의사결정역량 인프라 인프라관리경험3 운영경험3년 형상관리도구사용 서비스운영경험3 배포경험 프로그래밍언어 열정 성실 전공무관 해석 우선순위 동시 검증 전략적 촉진자 전략적사고 사고 역량보유자 동향 업적 획기적 관계형성 관련전공자 공무 개발경력2년 대용량 데이터 데이터 데이터 서비스개발 획득 엔지니어 디자인 괄목 성과 내신 리터 러시 수행업무 리더십역할경험 성과 4년대졸자 10년이상프로덕트 리더십 관리 열정 성실 성실전공무관 공무 개발경력2년 대용량데이터 데이터 최신기술 트렌드 디버깅스킬 대처 협력 팀워크 싸이클경험 프로젝트 분자격요건 단백 최신트렌드 룰수 부트 기숙사1년 사택 관적 최대5,000만원 27001 진심인분 취소 유니 있으신분웹퍼블리싱 급경험 여진 때적극적 포기 중간 타시스템과의 분필수자격요건 시멘 틱 지지 기반백 얼 분최소2년 라도 경우채용 중하나라 나라 모든 스플 집착 을경험 1년포함 모집 모집인원 위의조건 개발자로 자로 진심 개발신입 운개발환경 한협업 개발시 호출 3년웹 와이어 랫 폼 동의 액 회원 티비 스킬보유자 부서와의 전반 분수 맥락 징경력 분1년 공감대 안건 고유 누락 부문 설과 실질 전반적 전적 마케 기획이외 면접후결정 자간 소지자 그중 그중1년이상 중2가지 직무 설과인사이트 자타 현상태 기본적인영어문 해력 학분야 팀과의 신약 문제해결능력3 화한 보증금 일 변환 단기 수 있음 수있음 을2년간 분필수요건 지속적인 경험필요 인력 분이벤트로그설계 분개발자 원어 채용프로세스서류전형 2차면접 8년정도 결제 자격사항 지식1 기술과툴을사용해 작성가능6 경우면접 비지급 지급 대중교통 대중교통이동 이동 시외버스 기차 영수증 중 증빙시지원가능 빙시 연고지 기숙사 서의개발경험 졸업예정자 도포함 능력중 마인드모바일 문야 기기 기술서 경력기술서 외지역 력를 얼 아래3가지질문 답변포함 의안 수있음 있음 긍 의안정성 수조건 사업 센터 게신분 이상경력필요 연한 편입 경험자관련경력2 보유자임 졸업증 가지신분백 개발가능4 상황 이상4년이상 사용법 만큼 소양 52 지인 사용자대상 대상 분실무5년 위의 창성 아래의 편입신청 선착순 활용경력 컨셉 편입인원 타인 다회경 주요업무 희망연봉수준기재 사유 가지신분위 구성원 탈 누구 지환 팀원과의 페르 소나 이상경험자 때어려움 하면서변화 지환영 특별 공개 하이신분노션 분사용 모니터링하면서장애 자격제한 하면서 파이 아이디 면접시 해당과제 진행여부 과제진행 1차면접사전 내외부 위한아 분1차면접 관점 파악 한아 그룹 그룹사 자산구매 배포판무관 판무관 육아 의도 심층 급변 년이상실무경험자 휴직 대체 대체사용 구매 분실무경력2 부산 분필수경험 다양한도구들 한도 3년이상제품 성별 의학사이 최소한가지이상 인의 견 '최소1년이상 네트 하이 션에 상위 3년이상서비스 제외 한번 도움 제출 이유 바른 요원가능3년 주제 2회 주제무관 2회이상 접시 손끝 접시준비서류 과목 기본과목 준비 업종 트업 트업문화적응력 엔드개발경험 실무백 필요역량스택 이상백 게이 룰 창업멤버 시대 스타트업5,000억 7년내 멤버 3개이상 기반실무개발경력 기본적인이해 7급 명 이상유경험자 리드 800급 800 중최소1개 구조대신 대신 분기술스택 밍구현능력 제이 32 3개 기초적인프로 중가능 언어1개이상 언어중 내의 10년내의 기반하 반하 두려움 분전자 재현 깊이 의식 중이 수원 지역 이상12년이하 다음기술 6.0 무리 이상우대사항 외하나 당황 컴퓨터공학과학사이 개발자체 믿음 정신과 대우 배치 끝 과합리적 운영경험최소3 공통 공통기능 용성 업계최고대우 공학과 이력서상희망연봉 킹 마이 희망 기재필수학 1.8이상 관련업무경력5 자격요건 분성별무관 년이상기술스택 키오스 가능자7 환경하 1.8 경하 보유자5 활용가능자6 자백 분만4년이상 가지신분만8년 있으 있으신분 참여를 시분 학습의지5년 계신분 미들 안드로 실제최소3명 년이상경험 폴리오 분경력3년 미만 부심 필수사전질문 트웰브 년이상경력자관련 채용프로세스 근무장소 장소 채용절차 이상서비스 과의사소통 경우관련학과학 계속 기존 중이신분 활용가능자4 제로 가지신분통사항 분통 아시 보람 디버 흥미 초기 담장자 때담당자 구멍 일의 일의구멍 지금 단시간 내몰입 조직내 사이 2주단위 당업무 분최소4년 내재 분금융 개발충 간원활 자사 의협업경험 기개발 회사성공 숙지 초기스타트 마련 선아래 기대 분로그 경험2 렌치 베 디 부담 모두 기구 최소하나 기구축 모두가능 면접접차 직군과의 과의원활 접근 구두 소지 지속 셰 신분3 처리 이상가능 후배 분경력5년 7인이상규모 케이션역량 케이션설계 스터링 위경험 트개 721 전체 반응 분최소5년 코드적극적 의향 스를구 업환경이익숙하 실무진 진행 진행가능 1차인터뷰이후 010 5596 0870 승 대표 1콜챗 연락 연락주세요 세요 우리팀 지원이전 지원시지원서 시지 직전연봉 유의미 명시 확신 언제  최종합격순 실무인터뷰 아래질문 2차대면면접 우신분 최신스펙에대 욕심 분7년이상 힘 욕구 기타언어하나 이상7년이하 플리케이션 마음씨 페이지 주세 선호 구현경험 구조 팀인터뷰 자유양식 파일권장 혼자 공인 결과물 중심 본인만 외조직 컨트 를연결 인원 업자 고용 고용형태 급여변동 변동 서울시 서울시용산구이태원로 서류전형 1차기술과제 기술인터뷰 인터뷰 3.1 맵수 주2 3회 원리 업무수행가능자 2015 항목 취득 협의후결정학력 년이상자 이상자 항목중 이력서내 연봉필수기재 주5일 금 시차 근무일시 근무형태 근무조건 중최소한 분03 03 90 90일내발급 발급 서가 분04 04 05 이해4 이풍 머 풀이가능 1억원3 도환영 환영 롤 2기반 젠 간의 내가 12 유경험자 과적 방법 방식 24시간 미니 형태 확인 3기본지식 배경 혼합 외언어 부족 말 캐치 충족 충족가능 총백 요원 요건 년이상경력 어가 핀 셋 체감 피력 상대 고려 일정 의가 독성 기반웹 무엇인가 인가 즐거움분초 이상5년이하 가치 분우대 팀내외 내외 분풀 고성 갈구고성 내고 조건 분초대졸이상 관계 이공 분대졸 과적극적 53 실무5년차이 극적 팀내 자도구 도적 부서 타부서 컴퓨터 신분2 구분 지향 12년 내가작성 차장2 차이상의 3가능 자일 해당공고 기업내 2024 2024년2월 노 하우 처음 원의 원의성장 카 2022 2022년2월 한테스트 성격 전직 시유 대한 텍쳐에 수준 공용 과도 메인 글 각각 종류 통해 한어 필요하다 이번 15년이상 13 간의 대면 칭 채용 타파트 가실 5.0 십 가능자 셀 사양산 가실분 및브 릿 제한 입자 분경력8년 신청 8.1 보유필수 기타 웍 개발4 분지원가능 2017이후 2017 스펙에 저희 현재 인지 곳 신입기준연봉4,000 과의원할 브라 연봉범위 4000 6000 우징 리작성 에이 전시 10년이상 4,000 후협의 실시간 최소1개이상 신분4 7년이상3 몰 교 338 동702호 702 협의 면접 한시스템 3년이하 연봉3,000만원 중1가지이상 1시간 을이과정 군의 직군의사람 안주 처우 처우협의 입사예정일 예정일 확정 분1 9중하나이상 상태 결격 입장 6이상 급 용산 용산구 이태 원로 4년이상3 27 27길39 39 11 질문답변 3줄정도 줄 보신백 기반개발경력3 시점 지원자분의회사선택 3가지 분의 계신향후 향후 골 무엇 계신지식 정도 지원서 스합류에 무엇 계신부분 부분 우려 답변 2층 층 2차 최종합격 아래필수사 질문 합격 상세경 세경 1차코딩테스트 2명3년 안 드로 이드 제작 도로 태도로 우수 실패 대인 이야기 화면 이외 이내 갈분 애착 시도 16 인사 고와 절차 최대 이후 전형 이후단계 기능 필수적 육성 심사 스크 유저 분하나 중한개이상의사용 상의 그것 후재발방지 서버프로 어에 밍언어에 개인 이상전자 비즈니스요구사항 분3년이상 행복 급여 통합 추구 심도 후 개월 6개월사용기간 검토 여부 때행복 생성 10,000장이상 경우포함 런스문서 과오 언더 실내 264 265 수립 을 아래 달리 득능력 관행 템복잡도의 도의 운툴 중요 적용경험 연동 건강 불가 및운영  거시 와의 하다 문법사용 3사용경험 분신입 중요하다 성년자 을통 고민 고가 기재 지원시 어필 7년차 제학 논의 전공자최소 분관련경력3 저널 체제 치트 용화 소켓 그래밍경력 객체 지도 사람 세대 사람과의 분6년이상 양식 자유 입사 서류 지원서 입사지원서 소개서 코치 필요시 모습 중시 과의 해외여행 바탕 런경험 최소한가지 실제 중인 해본 분야 런스 문서 작성 분7 분3 실제운영중인서비스 \d개이상 2개이상 표준 케이션개발경험 경험보유 시장 순위 분개발경력4 풀이 불확실 예정자 남성 지원 기반개발업무경험 상용 한가지이상 15 15년 운영해 풀이경험 풀이 영해 다수 부설 처 중한가지이상언어 학과 계열 이제공 각종 숙자 졸업 중하나이상 프로토 콜 벡 라인 분시드 한자 보유한자 때문 덱싱 분6 소통하면 하면 오버 지양 적정 자기주 분빅 규격 빅 뷰 작용 상호 징 사항 능숙자 분6 1년이상인분 상응 인분 스타트 동작 키 텍 포함 마크 지경 지경험이 험이  명세 명세서에 서에 내부 외부 분학력 한가지 개발이익숙하 장애발생시 오너쉽이 무관 쉽이 이하 년이하 소프트 웨어 문화 5년이상7 리지 스토 재발 방지 시하 우선시하 움 분웹 겸비 분4년이상 이상근무 근무 사용기초 학력무관경력5 단위 동기 비동기이해 분5년이상 서의 발란 물 14 검색 사랑 분2년이상 분분업 주저 시스템 동종 때발 실무 분기본적인 등다량 다량 보신 유관 연봉5,000만원 분필수언어 요소 95 100 100점 점 레벨이상 스택대응 거부감 번 6 7 업계 연봉 000만원 연공5,000만원 스톡3,000만원 3,000 5,000 8,000 8,000만원 1억원이미지 억 이용해보신분 커뮤니케이션능력 모니터링경험 우선 분원 분원활 과제 데 항상 라면 도입 라면툴 거리낌 본인 최고 팔로 마음가짐 내용 해당내용 설파 전파 관련 컨테이너빌드 하나이상 소유자고객문제해결 의사 프로그래밍 분대 빌드 솔루션구축경험4 소스 이자 소유자 위해고객 적용 위해 4 년대 년대졸자 졸자 이상프로덕트 역할 기본 기쁨 불필요 본문제 워크 등딥러닝 계신 때 공자 80 역량보유 해당 중 중1개이상  습득 등 수 수 자 뮤 용량 이상경험 연봉5 동종업계경력6 우대 향상 실력다양 이해프레임 구성 자체 대외 최소5명이상 신분 티 브앱 시작 이용 보유 스킬 분경력 일 케이 원활 케이션 컴퓨팅환경 자격보유 자신 상상 야만족 최소 1가지이상 사용가능 분최소3년 자격 10,000 10,000 장 본분 프롬 프롬프 직접 값 값0.1 0.1 속 차이 하나하나 사전 허브 런 킨스 런분과 경우군필자 분과 무자 필자 개이상언어의사 분2 필수 필수사용자 사용자 툴 션 엔드개발경력3 2년이상 분백 엔드 형성 촉진 형상 의지소유자 수행 도구 가지신분대용량 그 활용 기술 케이션개발 워크개발 능숙하신분5 년이상보유 화가 초대 화가가능 생태계 운기술 10년 분기 분기초적 초적 해보 간 케이션능력 기초 러 스터 기반 참여 요구 역량 오픈 적인 3년이상 성공지표 지표 만 만5년이상 과장2 싸이클 가지신 등 선도 팀원 가능 종업 축하 사용 프레임 동 플리 보유자 성공 스스로 사용능력 언어 관심 새 직군 서비스'''
        word_tokens = kkma.nouns(kk)
        word_temp_1 = [word for word in word_tokens if word not in stop_words]
        result_a.append(', '.join(word_temp_1))
except:
    kk = None
data['a'] = result_a

result_b = []

for www in data.자격요건:
    word_temp_2 = re.findall(r'[a-zA-Z\s]+', www)
    result_b.append(', '.join(word_temp_2))

data['b'] = result_b

data['c'] = data['a'] + ', ' + data['b']

data.drop(columns=['a','b'], inplace = True)
data.drop(columns=['자격요건'], inplace = True)
data.rename(columns={'c':'자격요건'}, inplace = True)

In [19]:
data['자격요건'] = data['자격요건'].apply(lambda x: ','.join(word.strip() for word in x.split(',') if word.strip()))

In [20]:
data.to_excel('test.xlsx')

In [25]:
data

,기업명,업종분류,사원수,설립연도,기업형태,매출액,주소,직무,직급,이용하는기술스택/우대사항,자격요건
0,젭,데이터베이스 및 온라인정보 제공업,NaN,2021.0,NaN,NaN,서울특별시 강남구,소프트웨어 엔지니어,3,"웹소켓,메타버스,AWS",
1,서커스컴퍼니,응용 소프트웨어 개발 및 공급업,20.0,2012.0,중소기업,170563.0,서울특별시 강남구,소프트웨어 엔지니어,0,"소프트웨어,버전관리,AWS,EC,S,CF,Git","Node,Net,Nginx,Apache,SystemDesign,MySQL,Maria..."
2,블로코,응용 소프트웨어 개발 및 공급업,29.0,2014.0,중소기업,118214.0,경기도 성남시 분당구,소프트웨어 엔지니어,2,"블록체인,트랙트,프레임워크,암호화,spring,boot,nodejs,lua,Dock...",Golang
3,피매치,자연과학 및 공학 융합 연구개발업,NaN,2021.0,NaN,5286.0,서울특별시 성북구,소프트웨어 엔지니어,2,"자연어,LLM,Linux,Docker","AI,Python,Tensorflow,Pytorch"
4,만나플래닛,응용 소프트웨어 개발 및 공급업,134.0,2016.0,중소기업,1419493.0,서울특별시 구로구,소프트웨어 엔지니어,5,"프레임워크,트렌드,Rest API,API,Git,redmine",JAVA
...,...,...,...,...,...,...,...,...,...,...,...
1960,데브게이트,NaN,3.0,NaN,NaN,NaN,서울특별시 서초구,Problem Solver,0,"Slack,Notion,Figma,GitHub,TypeScript,Next,js,N...",AI
1961,데브게이트,NaN,3.0,NaN,NaN,NaN,서울특별시 서초구,Problem Solver,0,"TypeScript,Next,js,tailwind,css,zustand,NestJS...","ES,Typescript,DB,e,g,REST API,CRUD,Typescript,..."
1962,미스터카멜,NaN,10.0,NaN,NaN,0.0,서울특별시 용산구,데브옵스 엔지니어,0,"트래픽,핸들링,크롤링,프론트엔드,C,Selenium,Elasticsearch,Red...",OOP
1963,디에스랩글로벌,소프트웨어·솔루션·ASP,22.0,2020.0,중소기업,64095.0,서울특별시 영등포구,백엔드 개발자,0,"프레임워크,FastAPI,Django,AWS,Azure,Docker,Kubernet...",API
